<a href="https://colab.research.google.com/github/feiyoung/ReadPapers/blob/master/graphSAGE_NN_learn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 18.6 MB/s eta 0:00:00


## 1. A GraphSAGE graph neural network example for homogenious Graph's node classification

In [2]:
import dgl

dataset = dgl.data.CiteseerGraphDataset() ## get the Citerseer dataset
graph = dataset[0]
print(graph)

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Extracting file to /root/.dgl/citeseer_d6836239
Finished data loading and preprocessing.
  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Graph(num_nodes=3327, num_edges=9228,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(3703,), dtype=torch.float32)}
      edata_schemes={})


In [3]:
# 构建一个2层的GNN模型
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        # 实例化SAGEConve，in_feats是输入特征的维度，out_feats是输出特征的维度，aggregator_type是聚合函数的类型
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # 输入是节点的特征
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [4]:
node_features = graph.ndata['feat']
node_labels = graph.ndata['label']
train_mask = graph.ndata['train_mask']
valid_mask = graph.ndata['val_mask']
test_mask = graph.ndata['test_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [ ]:
def evaluate(model, graph, features, labels, mask):
    model.eval() # Sets the model to evaluation mode. In PyTorch, this is important because it disables operations like dropout,
    # which are usually active during training but should be turned off during evaluation.
    # This context manager ensures that during the following block of code, PyTorch won't track operations for gradient computation.
    # This is done for efficiency during evaluation since gradients are not needed.
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [ ]:
import torch
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=n_labels)
opt = torch.optim.Adam(model.parameters())

for epoch in range(20): ## For showing example, we set 20; user can set 300 for better performance
  # Puts the model in training mode. This is necessary because some layers,
  # like dropout or batch normalization, behave differently during training compared to evaluation.
    model.train() # set model to training mode: compute the grad autoly for parameters
    # 使用所有节点(全图)进行前向传播计算
    logits = model(graph, node_features)
    # 计算损失值
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # 计算验证集的准确度
    acc = evaluate(model, graph, node_features, node_labels, valid_mask)
    # 进行反向传播计算
    # Clears the gradients of all optimized tensors. This is necessary before computing the gradients for the next minibatch.
    opt.zero_grad()
    # Backward pass to compute the gradients of the loss with respect to the model parameters.
    loss.backward()
    # Updates the model parameters based on the computed gradients using the optimization algorithm (assumed to be stored in the opt variable).
    opt.step()
    # Prints the value of the training loss for the current epoch.
    if(epoch % 50 ==0):
      print(loss.item())

    # 如果需要的话，保存训练好的模型。本例中省略。


In [ ]:
print(loss) #
print(loss.item())
print(evaluate(model, graph, node_features, node_labels, valid_mask))


In [ ]:
## Test
evaluate(model, graph, node_features, node_labels, test_mask)

Note: some problems:
(1) how to increase the stacked layers with multiple SAGEConv layers
(2) how to obtain the node embedding after training?
(3) Add the output information for the loss on testing data
